In [1]:
import moku

from scope_utils import set_scope, get_waveform, get_trace_data, TraceData
from Peak_Search import ini_peak_search, reso_det, mod_peak_search,zero_measure,fir_measure
import pyvisa
import numpy as np
import numpy.typing as npt
import matplotlib.pyplot as plt
import nidaqmx
import h5py
import time
import tqdm
from typing import List, Tuple, Sequence
from utils_dBm import dBm_to_Vpp, Vpp_to_dBm, Vpp_to_Vrms, Vrms_to_watt
import csv

In [2]:
task = nidaqmx.Task()
task.ao_channels.add_ao_voltage_chan('Dev1/ao0','mychannel',0,5)
value = 4
task.start()
task.write(0)
task.stop()

In [3]:
from moku.instruments import MultiInstrument
from moku.instruments import ArbitraryWaveformGenerator, Oscilloscope
m = MultiInstrument('192.168.1.167',force_connect=True,platform_id=2)

In [4]:
# generate a signal that the Arbitrary Waveform Genenrator should generate on the output
period = 10
t =  np.linspace(0,period,10000)
de_spd = 1
fre = 8   # in Hz
# Sine wave with Exp decaying amplitude (can also use scipy.signal)
def input_wave(t,decay_speed,fre):
    v = np.exp(- decay_speed * t) * np.sin(2*np.pi*fre*t)
    return v
# Constant voltage trigger
def trig_sig(t,trig_vot):
    v = trig_vot
    return v

decay_wave = np.array([input_wave(x,de_spd,fre) for x in t])
trig_wave = np.array([trig_sig(x,0.5) for x in t])

amp_real = 0.2  # amplitude for AWG 

In [5]:
try:
    wg = m.set_instrument(1, ArbitraryWaveformGenerator)
    osc = m.set_instrument(2, Oscilloscope)
    # connections = []
    # m.set_connections(connections=connections)
    # print("connection clear")

    connections = [dict(source="Slot1OutA", destination="Slot2InA"),
                   dict(source="Slot1OutA", destination="Output1"),
                   dict(source="Input1", destination="Slot2InB"),]
    m.set_connections(connections=connections)

    m.set_frontend(1, "1MOhm", "DC", "0dB")
    osc.set_timebase(-1, 4 * period)


    osc.set_acquisition_mode(mode="Precision")
    osc.set_trigger(type="Edge",mode = "Normal", source="ChannelA", level= amp_real/2 * 0.9 )

    wg.burst_modulate(channel=1, trigger_source='External',trigger_mode='NCycle',burst_cycles=1,trigger_level=0.5)
    wg.generate_waveform(channel=1,sample_rate='Auto',
        lut_data=list(decay_wave), frequency = 1/4/period,
        amplitude = amp_real)
    wg.enable_output(1,enable=True)
    
    task.start()
    task.write(0)
    task.stop()
    task.start()
    task.write(value)
    task.stop()
    print("start scan")
    time.sleep(4*period)
    print("scan finished")

    # response = osc.save_high_res_buffer(comments="Test")
    # file_name = response["file_name"]
    # osc.download("persist", file_name, "~/high_res_data.li")

    
except Exception as e:
    raise e
finally:
    # Close the connection to the Moku device
    # This ensures network resources and released correctly
    wg.relinquish_ownership()



start scan
scan finished


In [6]:
task.start()
task.write(0)
task.stop()
task.start()
task.write(value)
task.stop()

In [9]:
task.start()
task.write(0)
task.stop()